In [1]:
import os

In [2]:
%pwd

'c:\\Users\\bedir\\OneDrive\\Masaüstü\\YMIR\\PROJECTS\\PYTHON\\AI\\MLOPS\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\\Chest-Canser-Mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\bedir\\OneDrive\\Masaüstü\\YMIR\\PROJECTS\\PYTHON\\AI\\MLOPS\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\\Chest-Canser-Mlflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_image_size:list
    params_lr:float
    params_include_top:bool
    params_weigths:str
    params_classes:int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self)->PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_lr=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weigths=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        return prepare_base_model_config
        

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [ ]:
class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig) -> None:
        self.config=config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weigths,
            include_top=self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path,model=self.model)
    
    @staticmethod
    def _prepare_full_model(model,classes,freeze_all,freeze_till,lr):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        
        pred = tf.keras.layers.Dense(units=classes,activation="softmax")(flatten_in)

        full_model = tf.keras.models.Model(inputs=model.input,outputs=pred)

        full_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=["accuracy"])

        full_model.summary()
        
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            lr=self.config.params_lr
        )
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)

    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)


In [ ]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e